In [1]:
from Bsor import make_bsor, Bsor, save_replay_to_file
from interpretMapFiles import create_map, Map

from Saber import Saber
from NoteCutter import NoteCutter
from noteManager import NoteManager
from scoreManager import ScoreManager

def calculate_score(map_data: Map, replay: Bsor) -> int:
    left_saber = Saber(0)
    right_saber = Saber(1)

    note_cutter = NoteCutter()
    note_manager = NoteManager(map_data, replay)
    score_manager = ScoreManager()

    print("# frames:", len(replay.frames))

    count = 0

    frames = replay.frames
    frames.sort(key=lambda x: x.time)

    for frame in frames:
        
        if count > 0:
            note_cutter.cut(left_saber, note_manager, replay)
            note_cutter.cut(right_saber, note_manager, replay)

        left_saber.manual_update(frame.left_hand, frame.time)
        right_saber.manual_update(frame.right_hand, frame.time)

        

        note_manager.update(frame)

        

        count += 1
        if count % 1000 == 0:
            print("Processed Frame:", count)

    events = note_manager.get_events()
    counter = 0

    for real in replay.notes:
        for predicted in events:
            if predicted.note_id == real.note_id and abs(predicted.spawn_time - real.spawn_time) < 0.01:
                if predicted.event_type != real.event_type or predicted.event_time != real.event_time:
                    # print(f"Predicted: {predicted.score.value}, Real: {real.score.value}. {predicted.cut.cutPoint} {real.cut.cutPoint}")
                    print(f"Time: {real.event_time} -> {predicted.event_time} Event: {real.event_type} -> {predicted.event_type}")
                    counter += 1
                elif hasattr(predicted, 'cut') and hasattr(real, 'cut'):
                    if round(predicted.cut.cutDistanceToCenter, 2) != round(real.cut.cutDistanceToCenter, 2):
                        counter += 1
                        print(f"Acc: {real.cut.cutDistanceToCenter:.2f} -> {predicted.cut.cutDistanceToCenter:.2f} Pre: {real.cut.beforeCutRating:.2f} -> {predicted.cut.beforeCutRating:.2f} After: {real.cut.afterCutRating:.2f} -> {predicted.cut.afterCutRating:.2f} ")
                    elif round(predicted.cut.cutPoint[2], 2) != round(real.cut.cutPoint[2], 2):
                        counter += 1
                        print(f"Cut point: {real.cut.cutPoint[2]:.2f} -> {predicted.cut.cutPoint[2]:.2f}")
                break
    
    print(f"Wrong events: {counter} / {len(replay.notes)}")
    replay.notes = events

    return score_manager.get_score()

MAP_PATH = '.\\generatedReplays\\1e199 (Cinco - Timbo)'
REPLAY_PATH = '.\\generatedReplays\\76561198157672038-ExpertPlus-Standard-C1CB957445C5BD92A8B4FEF3A2F9B99A7F775C0F.bsor'

with open(REPLAY_PATH, 'rb') as f:
    replay = make_bsor(f)
    f.close()

# replay.info.playerId = '19573' # PP bot ID

mapFile = create_map(MAP_PATH)

calculate_score(mapFile, replay)

print("Calculated score:", )
print("Actual score:", replay.info.score)

save_replay_to_file(replay, ".\\generatedReplays\\generatedwithscore.bsor")

# frames: 17506
NOT OK -166.02506911405172 4.289177417755127
NOT OK 116.25591139469789 13.171257019042969
Processed Frame: 1000
Processed Frame: 2000
NOT OK 5.514603257705415 28.29381561279297
Processed Frame: 3000
Processed Frame: 4000
NOT OK -151.36570988829467 65.49791717529297
Processed Frame: 5000
Processed Frame: 6000
Processed Frame: 7000
NOT OK -165.78335293776627 106.29113006591797
Processed Frame: 8000
NOT OK 16.778304139251922 120.31383514404297
Processed Frame: 9000
Processed Frame: 10000
Processed Frame: 11000
Processed Frame: 12000
Processed Frame: 13000
Processed Frame: 14000
Processed Frame: 15000
Processed Frame: 16000
Processed Frame: 17000
Time: 4.289177417755127 -> 4.289177417755127 Event: EventType.Good -> EventType.Bad
Time: 6.238884449005127 -> 6.224968433380127 Event: EventType.Good -> EventType.Good
Time: 10.276237487792969 -> 10.290153503417969 Event: EventType.Good -> EventType.Good
Time: 13.171257019042969 -> 13.171257019042969 Event: EventType.Good -> Event